In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

plt.style.use('seaborn-whitegrid')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
!git clone https://github.com/BigcontestParrot2020/csv.git

Cloning into 'csv'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [3]:
!pip install catboost
from catboost import CatBoostRegressor, Pool

     |████████████████████████████████| 66.1MB 65kB/s 


In [4]:
data = pd.read_csv('/content/csv/0921_ver.csv', encoding = 'utf-8', index_col = 0).iloc[:,1:]
data = data.dropna()

#data['구매횟수'] = data.취급액/data.판매단가

In [5]:
data['분'] = pd.to_datetime(data.index).strftime('%M')

# [CATBOOST] cateogorical features should be str
data['시'] = data['시'].astype(str)
data['프라임'] = data['프라임'].astype(str)
data['일'] = data['일'].astype(str)
data['휴일'] = data['휴일'].astype(str)

In [6]:
data['취급액_로그'] = np.log(data.취급액)
data = data.drop(columns = ['취급액'])

In [7]:
data.head()

,상품명,상품군,판매단가,월,일,시,소비층,요일,휴일,프라임,파트,분,취급액_로그
방송일시,,,,,,,,,,,,,
2019-01-01 06:00:00,테이트 남성 셀린니트3종,의류,39900,1,1,6,1,Tuesday,2,4,First,00,14.556972
2019-01-01 06:00:00,테이트 여성 셀린니트3종,의류,39900,1,1,6,1,Tuesday,2,4,First,00,15.290502
2019-01-01 06:20:00,테이트 남성 셀린니트3종,의류,39900,1,1,6,1,Tuesday,2,4,Middle,20,14.997851
2019-01-01 06:20:00,테이트 여성 셀린니트3종,의류,39900,1,1,6,1,Tuesday,2,4,Middle,20,15.754971
2019-01-01 06:40:00,테이트 남성 셀린니트3종,의류,39900,1,1,6,1,Tuesday,2,4,Later,40,15.713430


## Define Functions

In [8]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [9]:
model = CatBoostRegressor(iterations = 2000,
                          loss_function="RMSE",
                          eval_metric='RMSE',
                          learning_rate=0.1,
                          early_stopping_rounds=500,
                          random_seed = 123457,
                          )

In [22]:
clss = list(set(data.상품군))
month = list(set(data.월))

model_dict = {}
mape_dict = {}
for c in clss:
    print(f'===== start training {c} =====')
    train_df = data[data.상품군 == c]
    raw_x = train_df.drop(columns=['취급액_로그', '상품군'])
    raw_y = train_df[['취급액_로그','월']]

    cv_scores = {}
    cv_models = {}

    for m in month:

        try:
            test_x = raw_x[raw_x.월 == m].drop(columns = ['월'])
            test_y = raw_y[raw_y.월 == m].취급액_로그
            train_x = raw_x.drop(index = test_x.index, columns = ['월'])
            train_y = raw_y.drop(index = test_y.index).취급액_로그
        
            train_y = train_y.to_numpy().reshape(-1)
            test_y = test_y.to_numpy().reshape(-1)
            
            cat_features_train = np.where((train_x.dtypes != np.float)&(train_x.dtypes != np.int))[0]
            cat_features_test = np.where((test_x.dtypes != np.float)&(test_x.dtypes != np.int))[0]
    
            train_dataset = Pool(train_x, label = train_y, cat_features=cat_features_train)
            test_dataset = Pool(test_x, label = test_y, cat_features=cat_features_test)
            CBR = model.fit(train_dataset,  
                            eval_set = test_dataset,
                            use_best_model = True,      
                            verbose = False,
                            plot = False)
            predict_test = model.predict(test_x)

            test_y = np.exp(test_y)
            predict_test = np.exp(predict_test)

            mape = mean_absolute_percentage_error(test_y, predict_test)
            print(f'{m}월 MAPE:', mape)
            cv_scores.update({m:mape})
            cv_models.update({m:CBR})
      
        except :
            print(f'{m}월 MAPE: No Value')
            cv_scores.update({m:'no value'})
            cv_models.update({m:'no value'})

        mape_dict.update({c:cv_scores}) 
        model_dict.update({c:cv_models})

===== start training 건강기능 =====
1월 MAPE: 26.229875852721417
2월 MAPE: 38.6086695569613
3월 MAPE: 109.77440935842458
4월 MAPE: 41.46183534872427
5월 MAPE: 49.771708918077536
6월 MAPE: 28.55727087829742
7월 MAPE: 19.311686933885404
8월 MAPE: 63.92389762236921
9월 MAPE: 29.117430405672064
10월 MAPE: 34.36722477546941
11월 MAPE: 72.73699019694516
12월 MAPE: 24.04765440431769
===== start training 잡화 =====
1월 MAPE: 33.41168976170586
2월 MAPE: 33.28268807740537
3월 MAPE: 40.92530204474768
4월 MAPE: 34.758072832757186
5월 MAPE: 45.172552981164756
6월 MAPE: 39.19001898282245
7월 MAPE: 55.146240418132756
8월 MAPE: 80.2745819622941
9월 MAPE: 46.51585683460387
10월 MAPE: 63.080801717658666
11월 MAPE: 54.13392512153358
12월 MAPE: 73.86410396620273
===== start training 의류 =====
1월 MAPE: 55.45910451243897
2월 MAPE: 36.41806049836812
3월 MAPE: 44.18479102545897
4월 MAPE: 54.09854881323412
5월 MAPE: 48.43154402821225
6월 MAPE: 45.523565931919364
7월 MAPE: 38.03726521276672
8월 MAPE: 45.89266064533303
9월 MAPE: 85.5430237673413
10월 